# Candidate alignment (merging) 

**Necessary files:**

 - event_cands = dataframe with candidates and their tagged tweets
 - model = embeddings model trained on the event dataset (for the second merging step)
 

In [3]:
from merging_steps import merging_step1, merging_step2, merging_step3, merging_step4
from merging_steps import merge_indices, merge_transitively
import gensim  

import pickle
import os
import re
from tqdm import tqdm
from collections import defaultdict


# functions pickle_file and load_pickle merely help with storing pickled files in the event folders on drive
def pickle_file(file_name, file_to_dump):
    directory_path = os.getcwd() + "/../../../../"
    folder_name = file_name.split('_')[0]
    file_path = directory_path +  fr"Dropbox (CBS)/Master thesis data/Candidate Data/{folder_name}/{file_name}"
    with open(file_path, 'wb') as fp:
        pickle.dump(file_to_dump, fp)

def load_pickle(file_name):
    directory_path = os.getcwd() + "/../../../../"
    folder_name = file_name.split('_')[0]
    file_path = directory_path + fr"Dropbox (CBS)/Master thesis data/Candidate Data/{folder_name}/{file_name}"
    with open(file_path, "rb") as input_file:
        return pickle.load(input_file)

In [7]:
#load the GoogleNews 300dim model (fix path) - this is the model Hamborg uses
#model = gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\nikodemicek\Dropbox (CBS)\Master thesis data\GoogleNews-vectors-negative300.bin.gz', binary=True)
#model = load_pickle('moria_w2v_model')

#event candidates can be accessed using load_pickle function under the name [event]_short_cands
event_cands = load_pickle('tigray_cands')
event_cands.shape

(49727, 6)

### Merging step 1
  - In the first merging step, we merge two candidates if the head of each of their representative phrase is identical by string comparison.


In [8]:
# what_merged1 = defaultdict(list) # we do not need merging dictionary in the first step
event_cands_merged = event_cands.copy()

# function merge_indices takes a dataframe of candidates and indices to be removed (output of merging_step1 function)
event_cands_merged = merge_indices(event_cands_merged, merging_step1(event_cands_merged['candidates']))

100%|████████████████████████████████████████████████████████████████████████████| 49727/49727 [37:28<00:00, 22.12it/s]


Initial amount of candidates: 49727
Amount of candidates: 10190, after removing 39537 indices


In [9]:
pickle_file('tigray_cands_after1', event_cands_merged)

In [99]:
del event_cands_merged

### Merging step 2
 - In the second merging step, we merge two candidates if their set of phrases heads are similar ( we apply lower similarity  threshold for the candidates of the same type)
 - merged candidates are stored in the dictionary what_merged2

In [16]:
import numpy as np
from scipy.spatial.distance import cosine

def merging_step2(candidate_list, model, what_merged2):
    
    indices_to_remove = set()
    
    for up_cand_id in tqdm(range(len(candidate_list))):     
        up_cand = candidate_list[up_cand_id]
        
        up_cand_mean_vec = phrase_heads_avg_vector(up_cand[2],model)
        
        for low_cand_id in range(up_cand_id+1,len(candidate_list)): 
            low_cand = candidate_list[low_cand_id]
            #print(f'for index {candidate_list.index(longer_cand)} checking the index {candidate_list.index(cand)}')
            #if candidate_list[longer_cand][1] == candidate_list[cand][1]:
                #print(f'matching "{longer_cand}" with "{cand}"')
            low_cand_mean_vec = phrase_heads_avg_vector(low_cand[2],model)
            similarity = 1-cosine(up_cand_mean_vec,low_cand_mean_vec)
            if up_cand[3] != 'misc': #up_cand[3] == low_cand[3]:
                if similarity >= 0.75:
                    if similarity < 0.77:
                        print(f'matching "{up_cand[0]}" with "{low_cand[0]}" of the same type with {similarity} sim') 
                    indices_to_remove.add(low_cand_id)
                    what_merged2[up_cand[0].lower()].append(low_cand)
                        

            else:
                if similarity >= 0.85:
                    if similarity < 0.87:
                        print(f'matching "{up_cand[0]}" with "{low_cand[0]}" of diff type with {similarity} sim') 
                    indices_to_remove.add(low_cand_id)
                    what_merged2[up_cand[0].lower()].append(low_cand)



    return indices_to_remove, what_merged2

def nans(shape,dtype=float):
    nans = np.empty(shape,dtype)
    nans.fill(np.nan)
    return nans

def phrase_heads_avg_vector(phrase_set,model):
    phrase_head_vectors = []
    for phrase_head in phrase_set:    
        try:
            phrase_head_vectors.append(model.wv[phrase_head.lower()])
        # KeyError is raised when the word is not in the vocabulary (should not happen, since the model is trained on the data)    
        except KeyError:
            phrase_head_vectors.append(nans(model.wv.vector_size))
            pass
        # we get an attribute error when we use pretrained model instead of self-trained    
        except AttributeError:
            try:
                phrase_head_vectors.append(model[phrase_head.lower()])
            # KeyError is raised when the word is not in the vocabulary (should not happen if we build our vocabulary on top of pretrained model)        
            except KeyError:
                phrase_head_vectors.append(nans(model.vector_size))
                pass
            
    #phrase_head_vectors = [model[phrase_head] for phrase_head in phrase_set]
    if len(phrase_head_vectors) != 0:
        return np.nanmean(phrase_head_vectors,axis=0)
    else: 
        return np.NaN

In [33]:
#model = load_pickle('moria_w2v_model')
#model = gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\nikodemicek\Dropbox (CBS)\Master thesis data\GoogleNews-vectors-negative300.bin.gz', binary=True)


#model = gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\nikodemicek\Dropbox (CBS)\Master thesis data\wiki-news-300d-1M-subword.vec')
model = load_pickle('moria_ft_model')

what_merged2 = defaultdict(list) # we map the merged candidates in the dictionary
event_cands_merged2 = event_cands_merged[:2000].copy()

indices_to_remove, what_merged2 = merging_step2(event_cands_merged2['candidates'], model, what_merged2)

event_cands_merged2 = merge_indices(event_cands_merged2, indices_to_remove)


  0%|                                                                                 | 3/2000 [00:00<10:32,  3.16it/s]

matching "fire" with "campfire" of diff type with 0.8651066422462463 sim
matching "fire" with "a firefighter" of diff type with 0.8628710508346558 sim


  0%|▏                                                                                | 4/2000 [00:01<10:14,  3.25it/s]

matching "turkey" with "turkiye" of the same type with 0.7526381611824036 sim
matching "migrants" with "scientists and immigrants" of diff type with 0.8521818518638611 sim


  0%|▏                                                                                | 5/2000 [00:01<10:13,  3.25it/s]

...

matching "napier" with "kherter" of the same type with 0.7563822269439697 sim
matching "napier" with "a new method" of the same type with 0.769660234451294 sim
matching "napier" with "rocket" of the same type with 0.7589207887649536 sim


 97%|███████████████████████████████████████████████████████████████████████████▌  | 1939/2000 [05:16<00:00, 77.17it/s]

matching "cuba" with "ukraine" of the same type with 0.7570672035217285 sim
matching "ukraine" with "petrou ralli" of the same type with 0.7507758736610413 sim
matching "ukraine" with "by melissa eddy" of the same type with 0.7547239065170288 sim
matching "ukraine" with "ulla jelpke" of the same type with 0.7631782293319702 sim
matching "ukraine" with "eight fire brigades" of the same type with 0.7630745768547058 sim
matching "rocket" with "petros giannakouris" of diff type with 0.853737473487854 sim
matching "rochester" with "nayapara" of the same type with 0.7644994258880615 sim
matching "nayapara" with "dorchester" of the same type with 0.7519998550415039 sim
matching "the greek military elements" with "instructions" of diff type with 0.8608605265617371 sim


100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [05:16<00:00,  6.32it/s]

Initial amount of candidates: 2000
Amount of candidates: 821, after removing 1179 indices


In [34]:
what_merged2

defaultdict(list,
            {'greece': [('prime minister of greece',
               'minister',
               {'greece', 'minister'},
               'misc'),
              ('greeces', 'greeces', {'greeces'}, 'person-ne'),
              ('local anger as greece', 'anger', {'anger', 'greece'}, 'misc'),
              ('a calamity in greece',
               'calamity',
               {'calamity', 'greece'},
               'misc'),
              ('greecewho', 'greecewho', {'greecewho'}, 'loc-ne'),
              ('drowngreecethat',
               'drowngreecethat',
               {'drowngreecethat'},
               'misc'),
              (' enrichmentin greece',
               'enrichmentin',
               {'enrichmentin', 'greece'},
               'misc'),
              ('caritas greece', 'caritas', {'caritas', 'greece'}, 'group-ne'),
              ('unhcrgreece', 'unhcrgreece', {'unhcrgreece'}, 'group')],
             'refugees': [('refuge', 'refuge', {'refuge'}, 'misc'),
              

In [36]:
event_cands_merged2.head(20)

,candidates,cand_tags,cand_text,cand_len,cand_freq,string_len
0,"(greece, greece, {greece}, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Greece"",\n ...",greece,1,18978,6
1,"(refugees, refugees, {refugees}, misc)","[\n {\n ""id"": 1,\n ""text"": ""refugees"",\...",refugees,1,10656,8
2,"(fire, fire, {fire}, misc)","[\n {\n ""id"": 1,\n ""text"": ""fire"",\n ...",fire,1,7477,4
3,"(turkey, turkey, {turkey}, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Turkey"",\n ...",turkey,1,4986,6
4,"(migrants, migrants, {migrants}, misc)","[\n {\n ""id"": 1,\n ""text"": ""migrants"",\...",migrants,1,4666,8
5,"(lesbos, lesbos, {lesbos}, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Lesbos"",\n ...",lesbos,1,3202,6
6,"(europe, europe, {europe}, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Europe"",\n ...",europe,1,2516,6
7,"(euhomeaffairs, euhomeaffairs, {euhomeaffairs}...","[\n {\n ""id"": 1,\n ""text"": ""EUHomeAffai...",euhomeaffairs,1,2507,13
8,"(the greek islands, islands, {islands}, misc)","[\n {\n ""id"": 1,\n ""text"": ""the"",\n ...",the greek islands,3,2379,17
9,"(people, people, {people}, misc)","[\n {\n ""id"": 1,\n ""text"": ""people"",\n ...",people,1,2294,6


### Merging step 3
 - In the third merging step, we merge two candidates if their labeling phrases are similar 
 - merged candidates are stored in the dictionary what_merged3

In [37]:
what_merged3 = defaultdict(list)
event_cands_merged3 = event_cands_merged2.copy()

indices_to_remove, what_merged3 = merging_step3(event_cands_merged3, model, what_merged3)
event_cands_merged3 = merge_indices(event_cands_merged3, indices_to_remove)


what_merged3 = merge_transitively(what_merged3)
#what_merged3

100%|██████████████████████████████████████████████████████████████████████████████| 821/821 [00:00<00:00, 1076.03it/s]

Initial amount of candidates: 821
Amount of candidates: 757, after removing 64 indices


In [40]:
what_merged3

defaultdict(list,
            {'the greek islands': [('greek authorities',
               'authorities',
               {'authorities'},
               'misc'),
              ('crowded unsanitary conditions',
               'conditions',
               {'conditions'},
               'misc'),
              ('greek police', 'police', {'police'}, 'misc'),
              ('a greek immigrant', 'immigrant', {'immigrant'}, 'misc'),
              ('greek waters', 'waters', {'waters'}, 'misc'),
              ('greek officials', 'officials', {'officials'}, 'misc'),
              ('greek police move migrants',
               'move',
               {'migrants', 'move', 'police'},
               'misc'),
              ('greek soldiers', 'soldiers', {'soldiers'}, 'misc'),
              ('greek territory', 'territory', {'territory'}, 'misc'),
              ('greek pm promises', 'promises', {'pm', 'promises'}, 'misc'),
              ('greek govt', 'govt', {'govt'}, 'misc'),
              ('greek shephe

 ### Merging step 4
 - In the third merging step, we merge two candidates if their compound nouns are similar 
 - merged candidates are stored in the dictionary what_merged4

In [41]:
what_merged4 = defaultdict(list)
event_cands_merged4 = event_cands_merged3.copy()

indices_to_remove, what_merged4 = merging_step4(event_cands_merged4, model, what_merged4)
event_cands_merged4 = merge_indices(event_cands_merged4, indices_to_remove)

what_merged4 = merge_transitively(what_merged4)
#what_merged4

100%|████████████████████████████████████████████████████████████████████████████████| 757/757 [00:49<00:00, 15.21it/s]

Initial amount of candidates: 757
Amount of candidates: 163, after removing 594 indices


## Merge what_merged dictionaries

In [42]:
def merge_dicts(cands, wm2, wm3, wm4):
    # start with empty dictionary, the dictionary keys will be as many as candidates after 4th merging step
    merged = defaultdict(list)
    for cand in cands['cand_text']: #iterate over all cands
        # key's list of values will first contain all values from the same keys in the three merged dataframes
        merged[cand].extend(wm4[cand]) 
        merged[cand].extend(wm3[cand])
        merged[cand].extend(wm2[cand])
        
        # the values of the cand from 4th merging step can be keys in merging steps 3 & 2 
        for merged_cand3 in wm4[cand]: #iterate over all values in the list
            merged[cand].extend(wm3[merged_cand3[0]])
            merged[cand].extend(wm2[merged_cand3[0]])
            
            # the values of the cand from 3th merging step can be keys in merging step 2 
            for merged_cand2 in wm3[merged_cand3[0]]:
                merged[cand].extend(wm2[merged_cand2[0]])
            
    return merged

In [43]:
merged_dict = merge_dicts(event_cands_merged4, what_merged2, what_merged3, what_merged4)

In [44]:
for key,values in merged_dict.items():
    print(f'\n{key}: {set([val[0] for val in values])}')
event_cands_merged4.head(20)


greece: {'greekcitytimes', 'greek', 'prime minister of greece', 'a calamity in greece', 'greecewho', 'grenfell', 'greeces', 'drowngreecethat', ' enrichmentin greece', 'caritas greece', 'greekreporter', 'local anger as greece', 'greeks', 'unhcrgreece', 'greekisland'}

refugees: {'an asylum seeker', 'asylum seekers', 'constantinople', 'macedonians', 'a refugee', 'mercenaries', 'the dangers', 'refuge', 'failed asylum seekers', 'an illegalrefugee', 'therefugees', 'karen people', 'rohingya refugee camps unsays', 'the adventurers', 'rise in refugees', 'iraqi refugees tired', 'the inhabitants', '6refugees', 'refugeesgr', '90krefugees', 'people', 'refugees souls', 'few answers for refugees', 'iranian refugee omid masoumali'}

fire: {'al jazeera', 'samos', 'mytilene', 'mitillini island midilli', 'a small german firm', 'leros', 'our website', 'voices', 'high alert', 'the journey', 'ahval', 'greek refugee pushback campaign', 'many posted on amazon', 'croatia', 'mission in aegean sea', 'speedboat

,candidates,cand_tags,cand_text,cand_len,cand_freq,string_len
0,"(greece, greece, {greece}, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Greece"",\n ...",greece,1,18978,6
1,"(refugees, refugees, {refugees}, misc)","[\n {\n ""id"": 1,\n ""text"": ""refugees"",\...",refugees,1,10656,8
2,"(fire, fire, {fire}, misc)","[\n {\n ""id"": 1,\n ""text"": ""fire"",\n ...",fire,1,7477,4
3,"(turkey, turkey, {turkey}, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Turkey"",\n ...",turkey,1,4986,6
4,"(migrants, migrants, {migrants}, misc)","[\n {\n ""id"": 1,\n ""text"": ""migrants"",\...",migrants,1,4666,8
5,"(europe, europe, {europe}, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Europe"",\n ...",europe,1,2516,6
6,"(euhomeaffairs, euhomeaffairs, {euhomeaffairs}...","[\n {\n ""id"": 1,\n ""text"": ""EUHomeAffai...",euhomeaffairs,1,2507,13
7,"(fr bau und heimat, fr, {heimat, fr}, misc)","[\n {\n ""id"": 1,\n ""text"": ""für"",\n ...",fr bau und heimat,4,1819,17
8,"(germany, germany, {germany}, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Germany"",\n...",germany,1,1716,7
9,"(a humanitarian catastrophe, catastrophe, {cat...","[\n {\n ""id"": 1,\n ""text"": ""a"",\n ""l...",a humanitarian catastrophe,3,1602,26


In [19]:
#pickle_file('moria_short_cands_merged',event_cands_merged4)
pickle_file('moria_short_whatmerged2',what_merged2)
pickle_file('moria_short_whatmerged3',what_merged3)
pickle_file('moria_short_whatmerged4',what_merged4)

pickle_file('moria_short_cands_merged',event_cands_merged4)
pickle_file('moria_merged_dict',merged_dict)

#event_cands_merged4 = load_pickle("moria_short_cands_merged")


## Additional merging attempts 1: Using Affinity propagation clustering

In [ ]:
merged['greece']

In [ ]:
w2v_model = load_pickle('moria_ft_model')

In [ ]:
w2v_model.wv['greece']

In [ ]:
from sklearn.cluster import AffinityPropagation
import numpy as np


X = np.array([model.wv[word[1]] for word in event_cands_merged4['candidates']])


from sklearn.preprocessing import StandardScaler

scaled_x = StandardScaler().fit_transform(X)

from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca_x = pca.fit_transform(X)


clustering = AffinityPropagation(max_iter=500,random_state=42, verbose=True).fit(pca_x)
print(clustering)

In [ ]:
import pandas as pd
event_cands_merged4['labels'] = pd.Series(clustering.labels_)
event_cands_merged4['labels'].value_counts()

In [ ]:
event_cands_merged4[event_cands_merged4['labels']==]

## Additional merging attempts 2: Similarity using BERT embeddings

In [93]:
event_cands.head(10)

,candidates,cand_tags,cand_text,cand_len,cand_freq,string_len
0,"(greece, greece, {greece}, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Greece"",\n ...",greece,1,18978,6
1,"(refugees, refugees, {refugees}, misc)","[\n {\n ""id"": 1,\n ""text"": ""refugees"",\...",refugees,1,10656,8
2,"(fire, fire, {fire}, misc)","[\n {\n ""id"": 1,\n ""text"": ""fire"",\n ...",fire,1,7477,4
3,"(turkey, turkey, {turkey}, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Turkey"",\n ...",turkey,1,4986,6
4,"(migrants, migrants, {migrants}, misc)","[\n {\n ""id"": 1,\n ""text"": ""migrants"",\...",migrants,1,4666,8
5,"(lesbos, lesbos, {lesbos}, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Lesbos"",\n ...",lesbos,1,3202,6
6,"(europe, europe, {europe}, loc-ne)","[\n {\n ""id"": 1,\n ""text"": ""Europe"",\n ...",europe,1,2516,6
7,"(euhomeaffairs, euhomeaffairs, {euhomeaffairs}...","[\n {\n ""id"": 1,\n ""text"": ""EUHomeAffai...",euhomeaffairs,1,2507,13
8,"(the greek islands, islands, {islands}, misc)","[\n {\n ""id"": 1,\n ""text"": ""the"",\n ...",the greek islands,3,2379,17
9,"(people, people, {people}, misc)","[\n {\n ""id"": 1,\n ""text"": ""people"",\n ...",people,1,2294,6


In [46]:
#load bert embeddings (len = no. of unique tweets + len of unique candidates)
bert_embeddings = load_pickle('moria_bert_model')
bert_embeddings.shape

(267825, 768)

In [88]:
#candidate_list = all unique candidates
#model = bert embeddings (they start after the len of unique_tweets_df = for moria index 81327


def merging_step_bert(candidate_list, model):
    
    indices_to_remove = set()
    
    for up_cand_id in tqdm(range(len(candidate_list))):     
        up_cand = candidate_list[up_cand_id]
        
        up_cand_mean_vec = model[up_cand_id+81327]
        
        for low_cand_id in range(up_cand_id+1,len(candidate_list)): 
            low_cand = candidate_list[low_cand_id]
            #print(f'for index {candidate_list.index(longer_cand)} checking the index {candidate_list.index(cand)}')
            #if candidate_list[longer_cand][1] == candidate_list[cand][1]:
                #print(f'matching "{longer_cand}" with "{cand}"')
            low_cand_mean_vec = model[low_cand_id+81327]
            similarity = 1-cosine(up_cand_mean_vec,low_cand_mean_vec)
            if up_cand[3] != 'misc': #up_cand[3] == low_cand[3]:
                if similarity >= 0.80:
                    if similarity < 0.81:
                        print(f'matching "{up_cand[0]}" with "{low_cand[0]}" of the same type with {similarity} sim') 
                    indices_to_remove.add(low_cand_id)
                    what_merged2[up_cand[0].lower()].append(low_cand)
                        

            else:
                if similarity >= 0.80:
                    if similarity < 0.81:
                        print(f'matching "{up_cand[0]}" with "{low_cand[0]}" of diff type with {similarity} sim') 
                    indices_to_remove.add(low_cand_id)
                    what_merged2[up_cand[0].lower()].append(low_cand)



    return indices_to_remove, what_merged2


from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
rows_list = []
bert_corpus = event_cands['cand_text'][:10000]
for up_cand_id in tqdm(range(len(bert_corpus))):
    for low_cand_id in range(up_cand_id+1,len(bert_corpus)):
        sim = cosine_similarity(
            bert_embeddings[up_cand_id+81327].reshape(1,-1),
            bert_embeddings[low_cand_id+81327].reshape(1,-1)
        )
        dict1 = {}
        # get input row in dictionary format
        # key = col_name
        dict1.update({'text': bert_corpus[up_cand_id], 'text_compared':bert_corpus[low_cand_id], 'sim':sim[0][0]}) 

        rows_list.append(dict1)

sim_df = pd.DataFrame(rows_list)
sim_df

 36%|█████████████████████████▉                                               | 3550/10000 [2:35:02<4:41:42,  2.62s/it]


KeyboardInterrupt: 

In [89]:
sim_df

,text,text_compared,sim
487966,asylum seeker site,asylum seeker site,1.000000
10040,asylum seekers,asylum seekers,1.000000
30579,covid 19,covid 19,1.000000
273014,asylum seekers and refugees,refugees and asylum seekers,0.997586
115374,migrants and refugees,refugees and migrants,0.996311
...,...,...,...
359962,temporary home,greek police fire teargas,-0.034678
497944,a safe place,one million fire refugees,-0.043282
422980,course,refugees homeless after fire,-0.043481
125804,humane conditions,one million fire refugees,-0.043943


In [84]:
#sim_df.columns = sim_df.columns.str.strip()

sim_df.sort_values('sim', ascending=False,inplace=True)
sim_df.head(15)

list(sim_df['text'])

['asylum  seeker site',
 'asylum seekers',
 'covid 19',
 'asylum seekers and refugees',
 'migrants and refugees',
 'france and germany',
 'greece and turkey',
 'greece and italy',
 'that s',
 'huge fire',
 'migrant boats',
 'moria migrant camp',
 'the moria refugee camp',
 'greek refugee camp',
 'the aegean sea',
 'children',
 'turkish coast guard',
 'the greek islands',
 'european commission',
 'greek islands',
 'greek coast guard',
 'the greek island',
 'tens of thousands',
 'nothing',
 'new camp',
 'the greek islands',
 'migrant camp',
 'no one',
 'greek refugee camp',
 'fire',
 'the greek island',
 'african migrants',
 'illegal immigrants',
 'greek camps',
 'the greek border',
 'a huge fire',
 'the greek government',
 'the illegal immigrants',
 'refugee camp',
 'asylum seekers',
 'asylum  seekers',
 'greek authorities',
 'an asylum seeker',
 'greek borders',
 'these refugees',
 'the greek islands',
 'moria camp',
 'europe',
 'greek police',
 'the devastating fire',
 'the streets',


In [94]:
list(sim_df[(sim_df['text']=='fire') & (sim_df['sim']>0.80)]['text_compared'])

['fires',
 'flames',
 'this fire',
 'fire guts',
 'blaze',
 'a fire',
 'moria fire',
 'the fires',
 'the fire',
 'the flames',
 'the blaze',
 'the moria fire',
 'lesbos fire',
 'a massive fire',
 'a devastating fire',
 'the fire in moria',
 'fire at covid']

In [55]:
ids, wm_bert = merging_step_bert(event_cands['candidates'][:5000],bert_embeddings)

  0%|                                                                                         | 0/5000 [00:00<?, ?it/s]

matching "greece" with "a greek immigrant" of the same type with 0.8036025762557983 sim
matching "greece" with "greek refugees" of the same type with 0.8035529851913452 sim
matching "greece" with "turkey and greece" of the same type with 0.8070194721221924 sim
matching "greece" with "greece s refugees" of the same type with 0.8003939390182495 sim
matching "greece" with "the greek coastguard" of the same type with 0.8024135231971741 sim
matching "greece" with "greek immigrant" of the same type with 0.804611325263977 sim
matching "greece" with "greece and the eu" of the same type with 0.8083248734474182 sim
matching "greece" with "a greek chief executive" of the same type with 0.8086128830909729 sim


  0%|                                                                                 | 1/5000 [00:00<37:31,  2.22it/s]

matching "greece" with "the greek immigrants" of the same type with 0.8047448992729187 sim
matching "greece" with "hellenicpolice" of the same type with 0.8003621697425842 sim
matching "greece" with "greek coastguards" of the same type with 0.8045182228088379 sim
matching "greece" with "greece refugee camp" of the same type with 0.8024763464927673 sim
matching "greece" with "greek pushbacks" of the same type with 0.807540774345398 sim
matching "greece" with "greece  turkey" of the same type with 0.8010701537132263 sim
matching "greece" with "a greek refugee" of the same type with 0.8044360280036926 sim
matching "greece" with "greece s coast guard" of the same type with 0.8029263019561768 sim
matching "greece" with "greek asylum system" of the same type with 0.8067489266395569 sim
matching "greece" with "the greek economy" of the same type with 0.807970404624939 sim
matching "greece" with "greek officers" of the same type with 0.8084017634391785 sim
matching "refugees" with "a refugee c

  0%|                                                                                 | 2/5000 [00:00<36:35,  2.28it/s]

...

matching "some people" with "workers" of diff type with 0.8077688813209534 sim
matching "some people" with "a bit" of diff type with 0.8019205331802368 sim
matching "some people" with "a bunch" of diff type with 0.8092045187950134 sim
matching "some people" with "a photo" of diff type with 0.8056110739707947 sim


 16%|████████████▉                                                                  | 819/5000 [04:52<27:18,  2.55it/s]

matching "some people" with "their people" of diff type with 0.8025662302970886 sim
matching "some people" with "some reason" of diff type with 0.8045833110809326 sim


 16%|████████████▉                                                                  | 820/5000 [04:52<24:52,  2.80it/s]


KeyboardInterrupt: 

## Comparison of models in Merging step 2

In [30]:
#FIRST VARIATION FASTTEXT MODEL TRAINED ON MORIA DATA, THRESHOLDS 0.7 for same types and 0.8 otherwise

model1 = load_pickle('moria_fasttext_model')

what_merged2_1 = defaultdict(list) # we map the merged candidates in the dictionary
event_cands_merged2_1 = event_cands_merged.copy()

indices_to_remove, what_merged2_1 = merging_step2(event_cands_merged2_1['candidates'], model1, what_merged2_1)

event_cands_merged2_1 = merge_indices(event_cands_merged2_1, indices_to_remove)

matching "petersweden" with "a greco  swede" of the same type with 0.70261150598526 sim
matching "petersweden" with "voden" of diff type with 0.8057663440704346 sim


 37%|██████████████████████████▏                                           | 10827/28941 [15:53:45<22:23:11,  4.45s/it]

matching "herself on fire" with "the bullies in fire" of the same type with 0.9556370973587036 sim
matching "herself on fire" with "ambulances and fire" of the same type with 0.9501142501831055 sim
matching "herself on fire" with "velika kladua on fire" of the same type with 0.9653425216674805 sim
matching "herself on fire" with "methuenfire1691" of diff type with 0.8125804662704468 sim
matching "herself on fire" with "the cross fire naha" of the same type with 0.9626156091690063 sim
matching "herself on fire" with "firefighers" of the same type with 0.8559474945068359 sim
matching "herself on fire" with "appropriate to fire someone" of the same type with 0.9300042986869812 sim
matching "herself on fire" with "sutherland shire  rnp" of diff type with 0.8000339865684509 sim
matching "herself on fire" with "nashvillefiredept" of the same type with 0.7907846570014954 sim
matching "herself on fire" with "migrant boat catch fire" of the same type with 0.8617865443229675 sim
matching "hersel

matching "herself on fire" with "south ayrshire" of diff type with 0.8102748394012451 sim
matching "herself on fire" with "camp  fiweh" of the same type with 0.7016478180885315 sim
matching "herself on fire" with "scarlet begonias  fire" of the same type with 0.9519288539886475 sim
matching "herself on fire" with "slaithwaite fire service collections" of the same type with 0.8896652460098267 sim
matching "herself on fire" with "firecrotone" of the same type with 0.8888998031616211 sim
matching "herself on fire" with "moria fire archyde" of the same type with 0.9491018652915955 sim
matching "herself on fire" with "firefigthers" of the same type with 0.8497673273086548 sim
matching "herself on fire" with "laconia fire suspicious" of the same type with 0.9499112963676453 sim
matching "herself on fire" with "fire teargasfire" of the same type with 0.8224470615386963 sim
matching "herself on fire" with "welcome fire refugee cupcakes" of the same type with 0.9505058526992798 sim
matching "he

 37%|██████████████████████████▏                                           | 10828/28941 [15:53:49<22:35:12,  4.49s/it]

...

matching "20200701 cbc" with "total   126" of the same type with 0.748656153678894 sim
matching "20200701 cbc" with "bedtime2  8" of the same type with 0.7815717458724976 sim
matching "20200701 cbc" with "1993 y" of the same type with 0.7096225619316101 sim
matching "20200701 cbc" with "dubpl8  007" of the same type with 0.7006160020828247 sim


 38%|██████████████████████████▎                                           | 10869/28941 [15:56:29<22:38:08,  4.51s/it]

matching "the gender asymmetry" with "asymptomatic" of the same type with 0.7863169312477112 sim
matching "the gender asymmetry" with "an entire hybrid" of the same type with 0.7132734656333923 sim


 38%|██████████████████████████▎                                           | 10869/28941 [15:56:31<26:30:26,  5.28s/it]


KeyboardInterrupt: 

In [8]:
#SECOND VARIATION FASTTEXT PRETRAINED MODEL, THRESHOLDS 0.7 for same types and 0.8 otherwise

model2 = gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\nikodemicek\Dropbox (CBS)\Master thesis data\wiki-news-300d-1M-subword.vec')

what_merged2_2 = defaultdict(list) # we map the merged candidates in the dictionary
event_cands_merged2_2 = event_cands_merged.copy()

indices_to_remove, what_merged2_2 = merging_step2(event_cands_merged2_2['candidates'], model2, what_merged2_2)

event_cands_merged2_2 = merge_indices(event_cands_merged2_2, indices_to_remove)

  0%|                                                                                         | 0/2063 [00:00<?, ?it/s]C:\Users\nikodemicek\Documents\GitHub\refugee_project\merging_steps.py:104: RuntimeWarning: Mean of empty slice
  return np.nanmean(phrase_head_vectors,axis=0)


matching "greece" with "athens" of the same type with 0.7315918803215027 sim
matching "greece" with "italy" of the same type with 0.7035752534866333 sim
matching "greece" with "prime minister of greece" of diff type with 0.8634316921234131 sim
matching "greece" with "cyprus" of the same type with 0.7422357797622681 sim
matching "greece" with "bulgaria" of the same type with 0.7303168177604675 sim
matching "greece" with "macedonia" of the same type with 0.765284538269043 sim
matching "greece" with "albania" of the same type with 0.7289749383926392 sim


  0%|                                                                                 | 1/2063 [00:00<13:05,  2.62it/s]

matching "greece" with "a calamity in greece" of diff type with 0.8892802596092224 sim
matching "greece" with " enrichmentin greece" of diff type with 0.9999999972355171 sim
matching "greece" with "caritas greece" of diff type with 0.8874796032905579 sim
matching "refugees" with "migrants" of the same type with 0.7617825269699097 sim
matching "refugees" with "immigrants" of the same type with 0.7510162591934204 sim
matching "refugees" with "ministeriejenv thousands of refugees" of the same type with 0.885500696692674 sim
matching "refugees" with "a refugee" of the same type with 0.8043560981750488 sim


  0%|                                                                                 | 2/2063 [00:00<12:12,  2.82it/s]

...

matching "george bizos" with "chris howard" of the same type with 0.7667654655427973 sim


 95%|██████████████████████████████████████████████████████████████████████████▍   | 1968/2063 [06:17<00:02, 46.71it/s]

matching "i e" with "caliban s" of the same type with 0.7114444331370876 sim


100%|██████████████████████████████████████████████████████████████████████████████| 2063/2063 [06:18<00:00,  5.45it/s]

Initial amount of candidates: 2063
Amount of candidates: 1348, after removing 715 indices


In [ ]:
#THIRD VARIATION W2V PRETRAINED MODEL, THRESHOLDS 0.7 for same types and 0.8 otherwise

model3 = gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\nikodemicek\Dropbox (CBS)\Master thesis data\GoogleNews-vectors-negative300.bin.gz', binary=True)

what_merged2_3 = defaultdict(list) # we map the merged candidates in the dictionary
event_cands_merged2_3 = event_cands_merged.copy()

indices_to_remove, what_merged2_3 = merging_step2(event_cands_merged2_3['candidates'], model3, what_merged2_3)

event_cands_merged2_3 = merge_indices(event_cands_merged2_3, indices_to_remove)

In [ ]:
#FOURTH VARIATION W2V MODEL TRAINED ON MORIA DATA, THRESHOLDS 0.7 for same types and 0.8 otherwise

model4 = load_pickle('moria_w2v_model')

what_merged2_4 = defaultdict(list) # we map the merged candidates in the dictionary
event_cands_merged2_4 = event_cands_merged.copy()

indices_to_remove, what_merged2_4 = merging_step2(event_cands_merged2_4['candidates'], model4, what_merged2_4)

event_cands_merged2_4 = merge_indices(event_cands_merged2_4, indices_to_remove)

In [11]:
pickle_file('moria_wm2_1',what_merged2_1)
pickle_file('moria_wm2_2',what_merged2_2)
pickle_file('moria_wm2_3',what_merged2_3)
pickle_file('moria_wm2_4',what_merged2_4)

pickle_file('moria_after2_1', event_cands_merged2_1)
pickle_file('moria_after2_2', event_cands_merged2_2)
pickle_file('moria_after2_3', event_cands_merged2_3)
pickle_file('moria_after2_4', event_cands_merged2_4)

In [ ]:
what_merged2_1 = load_pickle('moria_wm2_1')
what_merged2_2 = load_pickle('moria_wm2_2')
what_merged2_3 = load_pickle('moria_wm2_3')
what_merged2_4 = load_pickle('moria_wm2_4')

event_cands_merged2_1 = load_pickle('moria_after2_1')
event_cands_merged2_2 = load_pickle('moria_after2_2')
event_cands_merged2_3 = load_pickle('moria_after2_3')
event_cands_merged2_4 = load_pickle('moria_after2_4')

model1 = load_pickle('moria_fasttext_model')
print('1st model loaded')
model2 = gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\nikodemicek\Dropbox (CBS)\Master thesis data\wiki-news-300d-1M-subword.vec')
print('2nd model loaded')
model3 = gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\nikodemicek\Dropbox (CBS)\Master thesis data\GoogleNews-vectors-negative300.bin.gz', binary=True)
print('3rd model loaded')
model4 = load_pickle('moria_w2v_model')
print('4th model loaded')

## Comparison of models in Merging step 3

In [ ]:
what_merged3_1 = defaultdict(list)
event_cands_merged3_1 = event_cands_merged2_1.copy()

indices_to_remove, what_merged3_1 = merging_step3(event_cands_merged3_1, model1, what_merged3_1)
event_cands_merged3_1 = merge_indices(event_cands_merged3_1, indices_to_remove)


what_merged3_1 = merge_transitively(what_merged3_1)

In [13]:
what_merged3_2 = defaultdict(list)
event_cands_merged3_2 = event_cands_merged2_2.copy()

indices_to_remove, what_merged3_2 = merging_step3(event_cands_merged3_2, model2, what_merged3_2)
event_cands_merged3_2 = merge_indices(event_cands_merged3_2, indices_to_remove)


what_merged3_2 = merge_transitively(what_merged3_2)

100%|█████████████████████████████████████████████████████████████████████████████| 1348/1348 [00:05<00:00, 267.42it/s]

Initial amount of candidates: 1348
Amount of candidates: 1233, after removing 115 indices


In [14]:
what_merged3_3 = defaultdict(list)
event_cands_merged3_3 = event_cands_merged2_3.copy()

indices_to_remove, what_merged3_3 = merging_step3(event_cands_merged3_3, model3, what_merged3_3)
event_cands_merged3_3 = merge_indices(event_cands_merged3_3, indices_to_remove)


what_merged3_3 = merge_transitively(what_merged3_3)


100%|█████████████████████████████████████████████████████████████████████████████| 1715/1715 [00:10<00:00, 168.80it/s]

Initial amount of candidates: 1715
Amount of candidates: 1589, after removing 126 indices


In [15]:
what_merged3_4 = defaultdict(list)
event_cands_merged3_4 = event_cands_merged2_4.copy()

indices_to_remove, what_merged3_4 = merging_step3(event_cands_merged3_4, model4, what_merged3_4)
event_cands_merged3_4 = merge_indices(event_cands_merged3_4, indices_to_remove)


what_merged3_4 = merge_transitively(what_merged3_4)


100%|███████████████████████████████████████████████████████████████████████████████| 308/308 [00:00<00:00, 837.48it/s]

Initial amount of candidates: 308
Amount of candidates: 289, after removing 19 indices


## Comparison of models in Merging step 4

In [ ]:
what_merged4_1 = defaultdict(list)
event_cands_merged4_1 = event_cands_merged3_1.copy()

indices_to_remove, what_merged4_1 = merging_step4(event_cands_merged4_1, model1, what_merged4_1)
event_cands_merged4_1 = merge_indices(event_cands_merged4_1, indices_to_remove)

what_merged4_1 = merge_transitively(what_merged4_1)

In [17]:
what_merged4_2 = defaultdict(list)
event_cands_merged4_2 = event_cands_merged3_2.copy()

indices_to_remove, what_merged4_2 = merging_step4(event_cands_merged4_2, model2, what_merged4_2)
event_cands_merged4_2 = merge_indices(event_cands_merged4_2, indices_to_remove)

what_merged4_2 = merge_transitively(what_merged4_2)

100%|██████████████████████████████████████████████████████████████████████████████| 1233/1233 [04:52<00:00,  4.21it/s]


Initial amount of candidates: 1233
Amount of candidates: 305, after removing 928 indices


In [18]:
what_merged4_3 = defaultdict(list)
event_cands_merged4_3 = event_cands_merged3_3.copy()

indices_to_remove, what_merged4_3 = merging_step4(event_cands_merged4_3, model3, what_merged4_3)
event_cands_merged4_3 = merge_indices(event_cands_merged4_3, indices_to_remove)

what_merged4_3 = merge_transitively(what_merged4_3)

100%|██████████████████████████████████████████████████████████████████████████████| 1589/1589 [09:17<00:00,  2.85it/s]


Initial amount of candidates: 1589
Amount of candidates: 452, after removing 1137 indices


In [19]:
what_merged4_4 = defaultdict(list)
event_cands_merged4_4 = event_cands_merged3_4.copy()

indices_to_remove, what_merged4_4 = merging_step4(event_cands_merged4_4, model4, what_merged4_4)
event_cands_merged4_4 = merge_indices(event_cands_merged4_4, indices_to_remove)

what_merged4_4 = merge_transitively(what_merged4_4)


100%|████████████████████████████████████████████████████████████████████████████████| 289/289 [00:10<00:00, 27.56it/s]

Initial amount of candidates: 289
Amount of candidates: 133, after removing 156 indices


## Final candidates and merged dictionaries

In [20]:
merged_dict1 = merge_dicts(event_cands_merged4_1, what_merged2_1, what_merged3_1, what_merged4_1)
#merged_dict2 = merge_dicts(event_cands_merged4_2, what_merged2_2, what_merged3_2, what_merged4_2)
#merged_dict3 = merge_dicts(event_cands_merged4_3, what_merged2_3, what_merged3_3, what_merged4_3)
#merged_dict4 = merge_dicts(event_cands_merged4_4, what_merged2_4, what_merged3_4, what_merged4_4)

In [ ]:
for key,values in merged_dict1.items():
    print(f'\n{key}: {set([val[0] for val in values])}')
event_cands_merged4_1.head(20)

In [20]:
for key,values in merged_dict2.items():
    print(f'\n{key}: {set([val[0] for val in values])}')
event_cands_merged4_2.head(20)

NameError: name 'merged_dict2' is not defined

In [ ]:
for key,values in merged_dict3.items():
    print(f'\n{key}: {set([val[0] for val in values])}')
event_cands_merged4_3.head(20)

In [ ]:
for key,values in merged_dict4.items():
    print(f'\n{key}: {set([val[0] for val in values])}')
event_cands_merged4_4.head(20)